# Pipeline:

1. Load the dataset.
2. Creating the data splits.
3. Data preprocessing and Augmentation.
4. Model Architecture and Training
6. Model Evaluation.

In [ ]:
pip install tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 48.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.0
    Uninstalling tensorboard-data-server-0.7.0:
      Successfully uninstalled tensorboard-data-server-0.7.0
  Attempting uninstall: google-auth-oauthlib
    Fo

In [ ]:
import tensorflow as tf

#verify the version
tf.__version__

'2.8.0'

In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Loading the Dataset

In [ ]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Set the path to the folder containing images
image_folder_path = '/content/drive/MyDrive/Landmarks-v1_1/'

# Define categories and landmarks
categories = ['Pagodas','Pyramids','Modern','Gothic','Mughal','Neoclassical']

landmarks2 = [   [ 'TianningTemplePagoda', 'ThienMuPagoda', 'GiantWildGoosePagoda', 'ShwedagonPagoda',    'FogongTemplePagoda'],[ 'Pyramid of Giza', 'Pyramid of Djoser', 'Santa Cecilia Acatitlan Pyramid',    'El Castillo, Chichen Itza', 'Louvre Pyramid'],[ 'Chrysler Building', 'Cathedral of Bras”lia',    'CCTV Headquarters', 'Hallgr¡mskirkja', 'eiffel'],['St.VitusCathedral',    'MilanCathedral', 'ChartresCathedral', 'CologneCathedral', 'Notre-DameCathedral'],[    'Taj Mahal', 'Tomb of Akbar', 'Jama Masjid', 'Tomb of I_timad-ud-Daulah',    'Bibi Ka Maqbara'],[ 'Buckingham Palace', 'Concertgebouw',    'Academy of Athens', 'Panth‚on', 'Ripon Building']]

# Define the image size
img_width, img_height = 255, 255

# Initialize empty lists to store the image, category, landmark, and image name data
image_data = []
categories_data = []
landmarks_data = []
image_names_data = []

# Loop through the categories and landmarks to read all the images and store them in the lists
for i, category in enumerate(categories):
    for j, landmark in enumerate(landmarks2[i]):
        # Set the image path and filename
        image_path = os.path.join(image_folder_path, category, landmark)
        extensions = ['jpg', 'jpeg', 'png']
        for extension in extensions:
            # Filter out files that do not have a valid image extension
            image_filenames = [f for f in os.listdir(image_path) if f.endswith(f".{extension}")]
            for image_filename in image_filenames:
                # Load and preprocess the image using Keras
                try:
                    image = load_img(os.path.join(image_path, image_filename), target_size=(img_width, img_height))
                    image_array = img_to_array(image).astype(np.uint8)

                    # Append the image data to the list
                    image_data.append(image_array)

                    # Append the category, landmark, and image name data to their corresponding lists
                    categories_data.append(category)
                    landmarks_data.append(landmark)
                    image_names_data.append(os.path.join(category, landmark, image_filename))
                except:
                    print(f"Failed to load {image_filename} in {image_path}")
                    continue

# Convert the category, landmark, and image name data to numpy arrays
categories_data = np.array(categories_data)
landmarks_data = np.array(landmarks_data)
image_names_data = np.array(image_names_data)

# Convert the image data to a numpy array
image_data = np.array(image_data)

# Print the shape of the image data, category data




Failed to load Bibi Ka Maqbara - 14.jpg in /content/drive/MyDrive/Landmarks-v1_1/Mughal/Bibi Ka Maqbara
Failed to load Buckingham Palace_19.jpg in /content/drive/MyDrive/Landmarks-v1_1/Neoclassical/Buckingham Palace
Failed to load Academy of Athens - 20.jpg in /content/drive/MyDrive/Landmarks-v1_1/Neoclassical/Academy of Athens


In [ ]:
print(categories_data.shape,landmarks_data.shape)

(417,) (417,)


## 2. Creating the data splits.

In [ ]:
import random

# Set the random seed for reproducibility
random.seed(42)
# Define the number of images to select for validation and test sets
num_val_images = 2
num_test_images = 3

# Initialize empty lists to store the image, category, and landmark data for the validation and test sets
val_image_data = []
val_categories_data = []
val_landmarks_data = []
test_image_data = []
test_categories_data = []
test_landmarks_data = []

# Initialize empty lists to store the indices of the images selected for the validation and test sets
val_indices = []
test_indices = []

# Loop through the categories and landmarks to select images for the validation set
for i, category in enumerate(categories):
    for j, landmark in enumerate(landmarks2[i]):
        # Set the indices of all the images in the current category and landmark
        image_indices = [k for k, c in enumerate(categories_data) if c == category and landmarks_data[k] == landmark]

        # Randomly select images for the validation set
        val_indices.extend(random.sample(image_indices, num_val_images))

        # Add the corresponding image, category, and landmark data to the validation set
        for idx in val_indices[-num_val_images:]:
            val_image_data.append(image_data[idx])
            val_categories_data.append(categories_data[idx])
            val_landmarks_data.append(landmarks_data[idx])

# Remove the validation images from the original dataset to create the remaining set
remaining_indices = list(set(range(len(categories_data))) - set(val_indices))
remaining_image_data = image_data[remaining_indices]
remaining_categories_data = categories_data[remaining_indices]
remaining_landmarks_data = landmarks_data[remaining_indices]

# Loop through the categories and landmarks to select images for the test set
for i, category in enumerate(categories):
    for j, landmark in enumerate(landmarks2[i]):
        # Set the indices of all the images in the current category and landmark
        image_indices = [k for k, c in enumerate(remaining_categories_data) if c == category and remaining_landmarks_data[k] == landmark]

        # Randomly select images for the test set
        test_indices.extend(random.sample(image_indices, num_test_images))

        # Add the corresponding image, category, and landmark data to the test set
        for idx in test_indices[-num_test_images:]:
            test_image_data.append(remaining_image_data[idx])
            test_categories_data.append(remaining_categories_data[idx])
            test_landmarks_data.append(remaining_landmarks_data[idx])

# Remove the test images from the remaining set to create the training set
train_indices = list(set(range(len(remaining_categories_data))) - set(test_indices))
train_image_data = remaining_image_data[train_indices]
train_categories_data = remaining_categories_data[train_indices]
train_landmarks_data = remaining_landmarks_data[train_indices]

# Convert the validation, test, and training data to numpy arrays
val_image_data = np.array(val_image_data)
val_categories_data = np.array(val_categories_data)
val_landmarks_data = np.array(val_landmarks_data)
test_image_data = np.array(test_image_data)
test_categories_data = np.array(test_categories_data)
test_landmarks_data = np.array(test_landmarks_data)
train_image_data = np.array(train_image_data)
train_categories_data = np.array(train_categories_data)
train_landmarks_data = np.array(train_landmarks_data)

## 3. Data preprocessing and Augmentation.

In [ ]:
tid = train_image_data
tcd = train_categories_data
tld = train_landmarks_data
vid = val_image_data
vcd = val_categories_data
vld = val_landmarks_data
sid = test_image_data
scd = test_categories_data
sld = test_landmarks_data

In [ ]:
from skimage.color import rgb2gray
from skimage.transform import rotate
from skimage.util import random_noise
from skimage.exposure import rescale_intensity

# augmenting the training images
tid2 = []
tcd2 = []
tld2 = []

for i in range(train_image_data.shape[0]):
    # original image
    tid2.append(rescale_intensity(train_image_data[i], in_range='image', out_range=(0,255)))
    tcd2.append(train_categories_data[i])
    tld2.append(train_landmarks_data[i])

    # image rotation
    tid2.append(rescale_intensity(rotate(train_image_data[i], angle=30, mode='edge'), in_range='image', out_range=(0,255)))
    tcd2.append(train_categories_data[i])
    tld2.append(train_landmarks_data[i])

    # image flipping (left-to-right)    
    tid2.append(rescale_intensity(np.fliplr(train_image_data[i]), in_range='image', out_range=(0,255)))
    tcd2.append(train_categories_data[i])
    tld2.append(train_landmarks_data[i])

    # image flipping (up-down) 
    tid2.append(rescale_intensity(np.flipud(train_image_data[i]), in_range='image', out_range=(0,255)))
    tcd2.append(train_categories_data[i])
    tld2.append(train_landmarks_data[i])

    # image noising
    tid2.append(rescale_intensity(random_noise(train_image_data[i], var=0.2), in_range='image', out_range=(0,255)))
    tcd2.append(train_categories_data[i])
    tld2.append(train_landmarks_data[i])


tid2 = np.array(tid2)
tcd2 = np.array(tcd2)
tld2 = np.array(tld2)


In [ ]:
# Define a dictionary to convert the category strings to integers
categories_dict = {'Gothic': 0, 'Modern': 1, 'Mughal': 2, 'Neoclassical': 3, 'Pagodas': 4, 'Pyramids': 5}

tcd2 = np.array([categories_dict[cat] for cat in tcd2])
print(tcd2, tcd2.shape)

# Convert the validation category data to integers
vcd2 = np.array([categories_dict[cat] for cat in vcd])
print(vcd2, vcd2.shape)

# Convert the test category data to integers
scd2 = np.array([categories_dict[cat] for cat in scd])
print(scd2, scd2.shape)

[4 4 4 ... 3 3 3] (1335,)
[4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3] (60,)
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3] (90,)


In [ ]:
tid2 = tid2.astype('float64') 
vid2 = vid.astype('float64')
sid2 = sid.astype('float64')

In [ ]:
print(tid2.shape,vid2.shape,sid2.shape)

(1335, 255, 255, 3) (60, 255, 255, 3) (90, 255, 255, 3)


In [ ]:
landmarks_dict = {}

count = 0
for landmark_group in landmarks2:
    for landmark in landmark_group:
        if landmark not in landmarks_dict:
            landmarks_dict[landmark] = count
            count += 1

print(landmarks_dict)
# Convert the training landmark data to integers
tld2 = np.array([landmarks_dict[landmark] for landmark in tld2])
print(tld2, tld2.shape)

# Convert the validation landmark data to integers
vld2 = np.array([landmarks_dict[landmark] for landmark in vld])
print(vld2, vld2.shape)

# Convert the validation landmark data to integers
sld2 = np.array([landmarks_dict[landmark] for landmark in sld])
print(sld2, sld2.shape)

{'TianningTemplePagoda': 0, 'ThienMuPagoda': 1, 'GiantWildGoosePagoda': 2, 'ShwedagonPagoda': 3, 'FogongTemplePagoda': 4, 'Pyramid of Giza': 5, 'Pyramid of Djoser': 6, 'Santa Cecilia Acatitlan Pyramid': 7, 'El Castillo, Chichen Itza': 8, 'Louvre Pyramid': 9, 'Chrysler Building': 10, 'Cathedral of Bras”lia': 11, 'CCTV Headquarters': 12, 'Hallgr¡mskirkja': 13, 'eiffel': 14, 'St.VitusCathedral': 15, 'MilanCathedral': 16, 'ChartresCathedral': 17, 'CologneCathedral': 18, 'Notre-DameCathedral': 19, 'Taj Mahal': 20, 'Tomb of Akbar': 21, 'Jama Masjid': 22, 'Tomb of I_timad-ud-Daulah': 23, 'Bibi Ka Maqbara': 24, 'Buckingham Palace': 25, 'Concertgebouw': 26, 'Academy of Athens': 27, 'Panth‚on': 28, 'Ripon Building': 29}
[ 0  0  0 ... 29 29 29] (1335,)
[ 0  0  1  1  2  2  3  3  4  4  5  5  6  6  7  7  8  8  9  9 10 10 11 11
 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23
 24 24 25 25 26 26 27 27 28 28 29 29] (60,)
[ 0  0  0  1  1  1  2  2  2  3  3  3  4  4  4  5  5  5  6 

In [ ]:
print(np.unique(vcd2))
print(np.unique(tcd2))
print(np.unique(scd2))
print(np.unique(vld2))
print(np.unique(tld2))
print(np.unique(sld2))

[0 1 2 3 4 5]
[0 1 2 3 4 5]
[0 1 2 3 4 5]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


## 4. Model Architecture and Training

 Final Selected Approach (out of 5 appraoches outlined in the report) : Training the model for landmarks data first for 30 classes (so, 30 neurons in the output layer). Then, using the same model (weights frozen) to train on the categories data but only swapping out the output layer so that the output is in the correct format.

### 4.1 : Model Architecture for landmarks data


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score

# Define hyperparameters
batch_size = 128
epochs = 50

# Define the EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(255, 255, 3))

#Freeze all the layers of EfficientNetB0 model
for layer in base_model.layers:
    layer.trainable = False

# Add a custom classifier and Dense layers on top of the base model
z = base_model.output
z = tf.keras.layers.GlobalAveragePooling2D()(z)
z = tf.keras.layers.Dense(1280, activation='relu')(z)
z = tf.keras.layers.Dropout(0.1)(z)
z = tf.keras.layers.Dense(540, activation='relu')(z)
z = tf.keras.layers.Dropout(0.1)(z)
z = tf.keras.layers.Dense(270, activation='relu')(z)
z = tf.keras.layers.Dropout(0.1)(z)
z = tf.keras.layers.Dense(135, activation='relu')(z)
z = tf.keras.layers.Dropout(0.1)(z)
z = tf.keras.layers.Dense(62, activation='relu')(z)
z = tf.keras.layers.Dropout(0.1)(z)
z = tf.keras.layers.Dense(30, activation='relu')(z)
predictions = tf.keras.layers.Dense(30, activation='softmax')(z)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.003)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


16719872/16705208 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 255, 255, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 255, 255, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D

### 4.2 Model Training for landmarks data

In [ ]:
# Define the model checkpoint callback to save weights at epoch with the best validation loss
checkpoint_filepath = 'landmark_weights.h5'
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

# Train the model
history = model.fit(tid2, tld2, batch_size=batch_size, epochs=epochs, validation_data=(vid2, vld2), callbacks=[checkpoint_callback])

# Load the weights of the best model
model.load_weights(checkpoint_filepath)
model.save('landmarks_model.h5')

Epoch 1/50
11/11 [==============================] - 25s 628ms/step - loss: 3.2733 - accuracy: 0.0794 - val_loss: 2.7693 - val_accuracy: 0.2000
Epoch 2/50
11/11 [==============================] - 3s 318ms/step - loss: 2.6276 - accuracy: 0.2135 - val_loss: 1.8160 - val_accuracy: 0.5000
Epoch 3/50
11/11 [==============================] - 4s 338ms/step - loss: 1.9479 - accuracy: 0.4112 - val_loss: 1.1247 - val_accuracy: 0.7500
Epoch 4/50
11/11 [==============================] - 4s 349ms/step - loss: 1.3411 - accuracy: 0.6195 - val_loss: 0.9069 - val_accuracy: 0.6667
Epoch 5/50
11/11 [==============================] - 4s 326ms/step - loss: 0.9611 - accuracy: 0.7109 - val_loss: 0.7451 - val_accuracy: 0.8000
Epoch 6/50
11/11 [==============================] - 4s 322ms/step - loss: 0.6902 - accuracy: 0.8067 - val_loss: 0.5950 - val_accuracy: 0.8167
Epoch 7/50
11/11 [==============================] - 3s 292ms/step - loss: 0.5213 - accuracy: 0.8419 - val_loss: 0.7193 - val_accuracy: 0.7833
Epoch

In [ ]:
# Make predictions on the validation data using the best model
y_pred = model.predict(vid2)
y_pred = np.argmax(y_pred, axis=1)

# Calculate metrics and print confusion matrix
print(classification_report(vld2, y_pred))
# print("Confusion Matrix:")
# print(confusion_matrix(vld2, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       0.33      0.50      0.40         2
           6       0.50      0.50      0.50         2
           7       1.00      0.50      0.67         2
           8       0.50      0.50      0.50         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       0.67      1.00      0.80         2
          14       1.00      1.00      1.00         2
          15       0.67      1.00      0.80         2
          16       1.00      1.00      1.00         2
          17       0.50    

### 4.3 : Model Architecture for categories data

#### Switching the output layer only and using the same model.

In [ ]:
y = model.layers[-2].output  # remove last Dense layer
predictions2 = tf.keras.layers.Dense(6, activation='softmax')(y)  # add new Dense layer with different output size
model2 = tf.keras.Model(inputs=model.input, outputs=predictions2)

# Freeze all layers except the last one
for layer in model2.layers[:-1]:
    layer.trainable = False

for layer in model.layers[-1:]:
    layer.trainable = True

optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.003)

model2.compile(optimizer=optimizer2, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model2.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 255, 255, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 255, 255, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 255, 255, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 257, 257, 3)  0           ['normalization[0][0]']    

### 4.4 : Model Training for categories data

In [ ]:
epochs2=20

checkpoint_filepath_2 = 'category_weights.h5'
checkpoint_callback_2 = ModelCheckpoint(filepath=checkpoint_filepath_2, save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

# Train the model
history2 = model2.fit(tid2, tcd2, batch_size=batch_size, epochs=epochs2, validation_data=(vid2 ,vcd2), callbacks=[checkpoint_callback_2])

# Load the weights of the best model
model2.load_weights(checkpoint_filepath_2)
model2.save('category_model.h5')

Epoch 1/20
11/11 [==============================] - 12s 456ms/step - loss: 16.2015 - accuracy: 0.0966 - val_loss: 13.7086 - val_accuracy: 0.1000
Epoch 2/20
11/11 [==============================] - 4s 325ms/step - loss: 11.1635 - accuracy: 0.1423 - val_loss: 10.0393 - val_accuracy: 0.2333
Epoch 3/20
11/11 [==============================] - 4s 349ms/step - loss: 8.2611 - accuracy: 0.2232 - val_loss: 7.2329 - val_accuracy: 0.3000
Epoch 4/20
11/11 [==============================] - 4s 358ms/step - loss: 6.0767 - accuracy: 0.2996 - val_loss: 5.0539 - val_accuracy: 0.3500
Epoch 5/20
11/11 [==============================] - 4s 329ms/step - loss: 4.1791 - accuracy: 0.3850 - val_loss: 3.3362 - val_accuracy: 0.4833
Epoch 6/20
11/11 [==============================] - 4s 330ms/step - loss: 2.9530 - accuracy: 0.4487 - val_loss: 1.8750 - val_accuracy: 0.5500
Epoch 7/20
11/11 [==============================] - 4s 341ms/step - loss: 1.8453 - accuracy: 0.5858 - val_loss: 0.9133 - val_accuracy: 0.7333
E

In [ ]:
# Make predictions on the validation data
y_pred2 = model2.predict(vid2)
y_pred2 = np.argmax(y_pred2, axis=1)

# Calculate metrics and print confusion matrix
print(classification_report(vcd2, y_pred2))
# print("Confusion Matrix:")
# print(confusion_matrix(vcd2, y_pred2))

# assuming your model is named 'model'
#model.save('best_model_2.h5')


              precision    recall  f1-score   support

           0       0.91      1.00      0.95        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10
           3       1.00      0.90      0.95        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10

    accuracy                           0.98        60
   macro avg       0.98      0.98      0.98        60
weighted avg       0.98      0.98      0.98        60



## 5. Evaluation on test set.

In [ ]:
y_pred_landmarks = model.predict(sid2) #sid2 contains test features
y_pred_landmarks = np.argmax(y_pred_landmarks, axis=1)

# Calculate metrics and print confusion matrix
print(classification_report(sld2, y_pred_landmarks)) #sld2 contains the test labels for landmarks

y_pred_categories = model2.predict(sid2) #sid2 contains test features
y_pred_categories = np.argmax(y_pred_categories, axis=1)

# Calculate metrics and print confusion matrix
print(classification_report(scd2, y_pred_categories)) #scd2 contains the test labels for categories

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         3
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         3
          11       0.75      1.00      0.86         3
          12       1.00      0.67      0.80         3
          13       1.00      0.67      0.80         3
          14       1.00      1.00      1.00         3
          15       1.00      1.00      1.00         3
          16       1.00      1.00      1.00         3
          17       1.00    

In [ ]:
# import matplotlib.pyplot as plt
# #Plot loss/accuracy graph

# fig = plt.figure(facecolor='white')

# pd.DataFrame(history.history).plot(figsize=(8, 5), xlim=[0, 50], ylim=[0, 1], grid=True, xlabel="Epoch",style=["g--", "g--.", "b-", "b-*"])

# plt.title('Model Performance - Landmarks')
# plt.ylabel('Loss/Accuracy')
# plt.legend(loc='best')

# plt.show()

In [ ]:
# #Plot loss/accuracy graph

# fig = plt.figure(facecolor='white')

# pd.DataFrame(history2.history).plot(figsize=(8, 5), xlim=[0, 10], ylim=[0, 1], grid=True, xlabel="Epoch",style=["g--", "g--.", "b-", "b-*"])

# plt.title('Model Performance - Categories')
# plt.ylabel('Loss/Accuracy')
# plt.legend(loc='best')

# plt.show()